# GR example

In [1]:
import rebound
import reboundx
import numpy as np
import ctypes

A simulation without GR

In [2]:
sim = rebound.Simulation()
sim.integrator = "ias15"
sim.add(m=1.)
sim.add(m=1e-6,a=0.01,e=0.1)
sim.move_to_com() # Moves to the center of momentum frame

sim.integrate(10.)
print("pomega = %f"%sim.calculate_orbits()[0].omega)

pomega = 0.000000


Now let's add GR

In [3]:
xs = reboundx.Extras(sim)
xs.add_gr()

and run for another 10 time units

In [4]:
sim.integrate(20.)
sim.calculate_orbits()[0].omega
print("pomega = %f"%sim.calculate_orbits()[0].omega)

pomega = 0.029886


As expected there was a pericenter precession!! 